# No rag Output process

In [6]:
import json
import os, re

In [23]:
query = r'^Write me a bio of: .+\n\n'
list_new_out = []
with open("../results/bio/bio_writing_no_rag_post.jsonl", "w") as out_f:
    with open("../results/bio/bio_writing_no_rag.jsonl", "r") as f:
        for line in f.readlines():
            out = json.loads(line)
            json.dump({"entity": out['entity'],
                        "generated_text" :re.split(query,out['generated_text'])[1]}, out_f)
            out_f.write('\n')
            


In [24]:
with open("../results/bio/bio_writing_no_rag_post.jsonl", "r") as f:
        for line in f.readlines():
            out = json.loads(line)
            print(out)

{'entity': 'Ludwig van Beethoven', 'generated_text': 'Ludwig van Beethoven (1770-1827) was a German composer and pianist who is widely considered one of the most important figures in the history of classical music. Born in Bonn, Germany, Beethoven began studying music at a young age and quickly demonstrated a prodigious talent. He went on to study with some of the leading composers of his day, including Joseph Haydn and Wolfgang Amadeus Mozart.\n\nDespite his early success as a composer and performer, Beethoven struggled with hearing loss throughout his life. This did not stop him from continuing to compose, however, and he went on to create some of his most famous works, including his symphonies, piano sonatas, and string quartets. Beethoven\'s music was revolutionary for its time, and it helped to pave the way for the Romantic movement in music.\n\nBeethoven\'s personal life was also marked by controversy and tragedy. He was known to be fiercely independent and had difficulty forming

# Generating a HF dataset from the FactScore Bio entities

In [2]:
import json
from datasets import load_dataset

In [4]:
with open("./prompt_entities.txt", 'r') as f_in:
    with open("./prompt_entities.jsonl", 'w') as f_out:
        for line in f_in.readlines():
            line = line.strip()
            cur_dict = {"input": "Write me a bio of: {}".format(line)}
            json.dump(cur_dict,f_out)
            f_out.write("\n")

ds = load_dataset('json', data_files=["./prompt_entities.jsonl"]

In [7]:
from datasets import load_dataset



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/lucas_dfki/.cache/huggingface/datasets/json/default-2ae1b026fdf5ee95/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# Exploring The Generations
## Non-Rag Llama2-Chat

In [9]:
import json
from pprint import pprint
from IPython.display import clear_output

In [10]:
very_rare = []
very_freq = []
rarity_set = set()


with open("../results/bio/results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        dct = json.loads(line)
        if "very rare" in dct['cat']:
            very_rare.append(dct)
        if "very freq" in dct['cat']:
            very_freq.append(dct)          
        rarity_set.add(dct['cat'][0])


print(rarity_set)
print("n v_rare: {}".format(len(very_rare)))
print("n v_freq: {}".format(len(very_rare)))

{'freq', 'very freq', 'medium', 'rare', 'very rare'}
n v_rare: 100
n v_freq: 100


In [11]:
# View rare outputs

for line in very_rare:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
#    a = input("for next")
#    if a == "x":
#        break
#    else:
#        continue


'Sun Wenguang'
('  Sun Wenguang (孙文光, born 1950) is a prominent Chinese dissident and human '
 'rights activist. He is known for his advocacy of democracy and civil rights '
 'in China, and has been imprisoned multiple times for his activism.\n'
 '\n'
 'Sun was born in the city of Changsha, Hunan Province, China. He studied at '
 'the Central Party School in Beijing, but was expelled in 1979 for his '
 'political views. He then worked as a farmer and a teacher, but was '
 'constantly harassed and persecuted by the authorities for his activism.\n'
 '\n'
 'In 1989, Sun was one of the leaders of the Tiananmen Square protests, which '
 'called for political reform and an end to corruption in China. After the '
 'crackdown on the protests, Sun was arrested and sentenced to 16 years in '
 'prison. He was released in 1995, but was immediately re-arrested and '
 'sentenced to another 10 years in prison for "inciting subversion of state '
 'power."\n'
 '\n'
 'Sun was released from prison in 200

In [12]:
# View very freq outputs

for line in very_freq:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
#    if a == "x":
#        break
#    else:
#        continue


'Nick Kyrgios'
('  Nick Kyrgios is a professional tennis player from Australia, born on April '
 '27, 1995, in Canberra, Australia. Here is a brief bio of the talented '
 'athlete:\n'
 '\n'
 'Early Life and Career:\n'
 'Nick Kyrgios was born to a Greek Cypriot father, Giorgos Kyrgios, and an '
 'Australian mother, Norlaila Kyrgios. He began playing tennis at a young age '
 'and quickly showed promise, winning the Australian Junior Open in 2011. '
 'Kyrgios turned professional in 2012 and made his ATP World Tour debut at the '
 'age of 17.\n'
 '\n'
 'Rise to Prominence:\n'
 'Kyrgios quickly gained attention for his powerful serve and aggressive '
 'playing style, which earned him the nickname "The Kid." He broke into the '
 'top 100 rankings in 2013 and reached the fourth round of Wimbledon that same '
 'year. In 2014, Kyrgios reached the quarterfinals of Wimbledon, defeating top '
 'players such as Rafael Nadal and Andy Murray along the way.\n'
 '\n'
 'Controversies and Injuries:\n'
 '

In [21]:
# Saving a subset of 25 samples from the 'very rare' and 'very freq' topics each

with open ("../results/bio/slice_results_llama2Chat_bio_prompts_ctxs.jsonl", "w") as f_out:
    for sample in very_rare[:25]:
        json.dump(sample,f_out)
        f_out.write("\n")
    for sample in very_freq[:25]:
        json.dump(sample,f_out)
        f_out.write("\n")
 

## With RAG Llama2-Chat7B


In [14]:
import json
from pprint import pprint
from IPython.display import clear_output

In [15]:
very_rare = []
very_freq = []
rarity_set = set()


with open("../results/bio/rag_results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        dct = json.loads(line)
        if "very rare" in dct['cat']:
            very_rare.append(dct)
        if "very freq" in dct['cat']:
            very_freq.append(dct)          
        rarity_set.add(dct['cat'][0])


print(rarity_set)
print("n v_rare: {}".format(len(very_rare)))
print("n v_freq: {}".format(len(very_rare)))

{'freq', 'very freq', 'medium', 'rare', 'very rare'}
n v_rare: 100
n v_freq: 100


In [16]:
# View rare outputs

for line in very_rare:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
    pprint(line['ctxs'])
#    if a == "x":
#        break
#    else:
#        continue

'Sun Wenguang'
(' Based on the provided documents, Sun Wenguang was born on August 26, 1934, '
 'in Rongcheng City, China. His father was a naval officer of the National '
 'Government. Sun joined the Navy in his early years and later became a '
 'professor of Physics at Shandong University in Jinan from 1982 until his '
 'retirement in 1994. Sun has been an activist and vocal critic of the Chinese '
 'government, being one of the original signatories of Charter 08, a '
 'pro-democracy manifesto published in 2008.\n'
 '\n'
 'Sun Xing, on the other hand, is a Hong Kong actor and Mandopop singer born '
 'in Guangzhou, China on October 16, 1963. He grew up in Beijing and '
 'immigrated to Hong Kong with his parents as a teenager. He started his '
 'acting career in Hong Kong and later shot to fame in Taiwan through a number '
 'of TV dramas, mostly portraying heroes.')
[{'id': '20828918',
  'score': '1.1635295',
  'text': 'Sun Wenguang Sun Wenguang (; born August 26, 1934) is an activist 

In [17]:
# View very freq outputs

for line in very_freq:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
#    if a == "x":
#        break
#    else:
#        continue
#    break


'Nick Kyrgios'
(' Sure, here is a bio of Nick Kyrgios based on the given documents:\n'
 '\n'
 'Nick Kyrgios is an Australian professional tennis player who was born on '
 'April 27, 1995, in Canberra, Australia. He is currently ranked world No. 35 '
 "in men's singles by the Association of Tennis Professionals (ATP). Kyrgios "
 'is of Greek Orthodox Christian descent and always wears a gold necklace with '
 'a cross on it.\n'
 '\n'
 'Kyrgios began playing tennis at a young age and quickly showed promise as a '
 'promising basketball player. He represented the Australian Capital Territory '
 'and Australia in his early teens before deciding to focus solely on tennis '
 'when he was 14 years old. Two years later, he gained a full scholarship at '
 'the Australian Institute of Sport, where he was able to further develop his '
 'tennis skills.\n'
 '\n'
 'In 2013, Kyrgios relocated his training base from Canberra to Melbourne '
 'Park. He has since achieved significant success on the ATP to

In [20]:
# Saving a subset of 25 samples from the 'very rare' and 'very freq' topics each

with open ("../results/bio/slice_rag_results_llama2Chat_bio_prompts_ctxs.jsonl", "w") as f_out:
    for sample in very_rare[:25]:
        json.dump(sample, f_out)
        f_out.write("\n")
    for sample in very_freq[:25]:
        json.dump(sample, f_out)
        f_out.write("\n")
 